In [1]:
!pip install torch torchdata transformers datasets

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/74/07/edce54779f5c3fe8ab8390eafad3d7c8190fce68f922a254ea77f4a94a99/torch-2.1.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchdata from https://files.pythonhosted.org/packages/f4/0c/b0c538d9d439e44f639a35c37737db37f3ea816ac5a0b4e6c7b7ba941c6e/torchdata-0.7.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/192.3 MB 1.4 MB/s eta 0:02:21
   ---------------------------------------- 0.1/192.3 MB 2.1 MB/s eta 0:01:30
   ---------------------------------------- 0.3/192.3 MB 2.7 MB/s eta 0:01:11
   ---------------------------------------- 0.5/192.3 MB 3.7 MB/s eta 0:00:52
   ---------------------------------------- 0.7/192.3 MB 3.2 MB/s eta 0:01:00
   ---------------------------------------- 0.9/192.3 MB 3.6 MB/s eta 0:00:53
   ---------------------------------------- 1

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer,GenerationConfig

## summarize data without prompt engineering

In [3]:
from datasets import load_dataset

dataset = load_dataset("knkarthick/dialogsum")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/tusha/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# example
indices = [40,200]

dash_line = '-'.join('' for x in range(100))

for i,index in enumerate(indices):
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print('INPUT DIALOGUE :')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY :')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE :
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY :
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
E

In [6]:
# use FLAN-T5 model
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')

c:\Users\tusha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tusha\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base',use_fast=True)

In [14]:
# example of tokenization
sentence = "What time is it ?"
sentence_encoded = tokenizer(sentence,return_tensors='pt')

sentence_decoded = tokenizer.decode(sentence_encoded["input_ids"][0],skip_special_tokens=True)

print('ENCODED SENTENCE: ' + str(sentence_encoded["input_ids"][0].tolist()))
print('DECODED SENTENCE: ' + sentence_decoded)

ENCODED SENTENCE: [363, 97, 19, 34, 3, 58, 1]
DECODED SENTENCE: What time is it?


In [15]:
# testing model
for i,index in enumerate(indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue,return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50  # tuning parameters
        )[0],
        skip_special_tokens=True
    )
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print('INPUT PROMPT :')
    print(dialogue)
    print(dash_line)
    print('BASELINE HUMAN SUMMARY :')
    print(summary)
    print(dash_line)
    print('MODEL OUTPUT - WITHOUT PROMPT ENGINEERING :\n' + output )


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT :
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY :
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL OUTPUT - WITHOUT PROMPT ENGINEERING :
Person1: It's ten to nine.
---------------------------------

## summarize dialogue with an instruction prompt

### zero shot inference with an instruction prompt

in order to instruct the model to perform a task - summarize a dialogue - you can take the dialogue and convert it into an instruction prompt . This is often called zero shot inference .

In [16]:
for i,index in enumerate(indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Summarize the following conversation .
    {dialogue}
    Summary :
    """
    # input constructed prompt instead of dialogue.
    inputs = tokenizer(prompt,return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50  # tuning parameters
        )[0],
        skip_special_tokens=True
    )
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print('INPUT PROMPT :')
    print(prompt)
    print(dash_line)
    print('BASELINE HUMAN SUMMARY :')
    print(summary)
    print(dash_line)
    print('MODEL OUTPUT - ZERO SHOT :\n' + output )

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT :

    Summarize the following conversation .
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    Summary :
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY :
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL OUTPUT - ZERO SHOT :
The train 

### here we are trying to change the prompt so as to get better answer

This is called Prompt engineering

In [17]:
for i,index in enumerate(indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Dialogue
    {dialogue}
    What was going on ?
    """
    # input constructed prompt instead of dialogue.
    inputs = tokenizer(prompt,return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50  # tuning parameters
        )[0],
        skip_special_tokens=True
    )
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print('INPUT PROMPT :')
    print(prompt)
    print(dash_line)
    print('BASELINE HUMAN SUMMARY :')
    print(summary)
    print(dash_line)
    print('MODEL OUTPUT - ZERO SHOT :\n' + output )

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT :

    Dialogue
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    What was going on ?
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY :
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL OUTPUT - ZERO SHOT :
The train is about to leave To

## summarize dialogue with one shot and few shot inference
One shot and few shot inference are the practice of providing an LLM with either one or more full examples of prompt-response pairs that match your task - before your actual prompt that you want completed. This is called "in-context learning" and puts your model into a state that understands your specific task.

In [18]:
def make_prompt(example_indices,example_indices_to_summaize):
    prompt = ''
    for index in example_indices:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        prompt+= f"""
Dialogue:
{dialogue}
What was going on?
{summary}
"""
    dialogue = dataset['test'][example_indices_to_summaize]['dialogue']
    prompt+= f"""
Dialogue:
{dialogue}
What was going on?
"""
    return prompt

In [19]:
example_indices = [40]
example_indices_to_summaize = 200
one_shot_prompt = make_prompt(example_indices,example_indices_to_summaize)
print(one_shot_prompt)


Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a 

In [20]:
summary = dataset['test'][example_indices_to_summaize]['summary']

inputs = tokenizer(one_shot_prompt,return_tensors='pt')
output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50  # tuning parameters
        )[0],
        skip_special_tokens=True
    )
print(dash_line)
print('BASELINE HUMAN SUMMARY :')
print(summary)
print(dash_line)
print('MODEL OUTPUT - ONE SHOT :\n' + output )

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY :
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL OUTPUT - ONE SHOT :
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


### few shot inference

In [21]:
example_indices = [40,80,120]
example_indices_to_summaize = 200
few_shot_prompt = make_prompt(example_indices,example_indices_to_summaize)
print(few_shot_prompt)


Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue:
#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. May, 

In [22]:
summary = dataset['test'][example_indices_to_summaize]['summary']

inputs = tokenizer(few_shot_prompt,return_tensors='pt')
output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50  # tuning parameters
        )[0],
        skip_special_tokens=True
    )
print(dash_line)
print('BASELINE HUMAN SUMMARY :')
print(summary)
print(dash_line)
print('MODEL OUTPUT - FEW SHOT :\n' + output )

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY :
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL OUTPUT - FEW SHOT :
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.
